In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Agents
The Agent interface provides the flexibility for such applications. An agent has access to a suite of tools, and determines which ones to use depending on the user input. Agents can use multiple tools, and use the output of one tool as the input to the next.

In [ ]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain import OpenAI

In [ ]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world",
    ),
]

### Zero Shot React
This agent uses the ReAct framework to determine which tool to use based solely on the tool's description. Any number of tools can be provided. Requires that a description is provided for each tool.

This is the most general purpose action agent.

In [ ]:
llm = OpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
agent_chain.run(input="Give me a list of the most popular movies in 2023.")

### Chat Zero Shot React
This is a variation of Zero Shot optimized for chat models

In [ ]:
llm = ChatOpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
agent_chain.run(input="Give me a list of the most popular movies in 2023.")

In [ ]:
agent_chain.run(
    input="what are some good dinners to make this week, if i like thai food?"
)

### Chat conversational react
This agent is designed to be used in conversational settings. The prompt is designed to make the agent helpful and conversational. It uses the ReAct framework to decide which tool to use, and uses memory to remember the previous conversation interactions.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

In [ ]:
agent_chain.run(
    input="Hi,how are you? I want to go to the cinema on this weekend. Can you recommend a good movie?"
)

In [ ]:
agent_chain.run(input="Can you give me the synopsis of them?")

In [ ]:
memory.load_memory_variables({})

### Plan and execute agent
Plan and execute agents accomplish an objective by first planning what to do, then executing the sub tasks. This idea is largely inspired by BabyAGI and then the "Plan-and-Solve" paper.

The planning is almost always done by an LLM.

The execution is usually done by a separate agent (equipped with tools).

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [ ]:
from langchain.utilities import WikipediaAPIWrapper

search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
wikipedia = WikipediaAPIWrapper()
tools = [
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for searching information about everything",
    ),
]

In [ ]:
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.agents.agent_toolkits import FileManagementToolkit
from tempfile import TemporaryDirectory

import os

dir_name = "agent_output"
current_directory = os.getcwd()
full_path = os.path.join(current_directory, dir_name)
if not os.path.exists(full_path):
    os.makedirs(full_path)


toolkit = FileManagementToolkit(root_dir=dir_name).get_tools()
tools.extend(toolkit)

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [ ]:
agent.run(
    "I need you to create a csv file for 10 cat breeds, with the columns name and origin"
)